In [1]:
import requests
import json
from pprint import pprint

import os, sys
sys.path.insert(1, os.path.dirname(sys.path[0])) 

In [2]:
from deploy import text_processor

In [3]:
username = 'admin'
password = os.getenv('PGPASSWORD')

# end point of Elasticsearch
elastic_endpoint = 'http://13.79.79.34:9094/elasticsearch/'
# elastic_endpoint = "http://134.0.107.93:9094/elasticsearch/"


In [4]:
def search(text:str)->object:
    q = {
      "from": 0,
      "size": 100,
      "timeout": "2s",
      "_source": [ "obj_id", "title","announce","uannounce", "link_title", "url" ],
      "query": {
        "match": {
          "lemmatized_text": {
            "query": text
          }
        }
      }    
    }
    
    rjson = {}
    r = requests.post(f'{elastic_endpoint}articles/_search', 
                      headers = {'Content-Type': 'application/json; charset=UTF-8'}, 
                      auth=(username,password),
                      data= json.dumps(q))
#                       data=query_str.encode('utf-8'))
    try:
        rjson=r.json()
        if rjson.get('errors') is not None:
            pprint(rjson)
            print(type(rjson))
    except:
        pprint(r)

    return rjson

def build_links(r:object)-> str:
    """ Builds html links from the object returned by Elastic.
    """
    if r is None: 
        return ''
    
    h = r.get('hits')
    if h is None: 
        return ''
    
    hh = h.get('hits',[])
    
    links = [f"""
    <span style="width:100px; display:inline-block">{o['_score']}</span> 
    <a href="{o['_source']['url']}" 
    title="{o['_source']['announce']}" 
    target="_blank">{o['_source']['link_title']}</a>
    <!-- <span style="margin-left:20px;">aaaaaaaa{o['_source']['announce']}</span> -->
    <br>
    """ for o in hh]
    
    return ''.join(links)


In [7]:
from ipywidgets import widgets, Layout
from IPython.display import display


txt = widgets.Textarea("слон",layout=Layout(width='100%', height='200px'))


btn = widgets.Button(description='search')
btn.on_click(on_btn_clicked)
display(txt)
display(btn)

# on_btn_clicked(5)


Textarea(value='слон', layout=Layout(height='200px', width='100%'))

Button(description='search', style=ButtonStyle())

In [6]:
def on_btn_clicked(e):
    text = txt.value
    print(f'new val={text}')
    o = text_processor.process_text(text)
    pprint(o)
    r = search(o['lemmatized_text'])
    html = build_links(r)
#     display(HTML(html))
    print(html)
print(444)

444
